# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [16]:
# import libraries
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
%matplotlib inline

import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk import pos_tag

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

In [17]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('DisasterResponse', engine)  
X = df.message
Y = df.drop(labels=["id","message","original","genre"],axis=1)

### 2. Write a tokenization function to process your text data

In [18]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
def tokenize(text):
    
    # remove punctuation and tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    token = tokenizer.tokenize(text.lower())

    # remove stop words:
    stopwords_english = stopwords.words("english")
    tokens = [word for word in token if word not in stopwords_english]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [20]:
pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(KNeighborsClassifier(n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
model = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
def display_results(y_true, y_pred):
    for column in Y.columns:
        print("\n", column, "\n")
        print(classification_report(y_true[column], y_pred[:,0]))
        
y_pred = model.predict(X_test)
display_results(Y, y_test, y_pred)


 related 

             precision    recall  f1-score   support

          0       0.74      0.11      0.20      1245
          1       0.78      0.99      0.87      3998

avg / total       0.77      0.78      0.71      5243


 request 

             precision    recall  f1-score   support

          0       0.95      0.04      0.08      4352
          1       0.17      0.99      0.30       891

avg / total       0.82      0.20      0.12      5243


 offer 

             precision    recall  f1-score   support

          0       1.00      0.04      0.07      5219
          1       0.00      1.00      0.01        24

avg / total       1.00      0.04      0.07      5243


 aid_related 

             precision    recall  f1-score   support

          0       0.94      0.06      0.11      3079
          1       0.43      0.99      0.60      2164

avg / total       0.73      0.45      0.31      5243


 medical_help 

             precision    recall  f1-score   support

          0       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
parameters = {"clf__estimator__n_neighbors":[5,50,100]}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
display_results(y_test, y_pred)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_neighbors=5 ...................................
[CV] .................... clf__estimator__n_neighbors=5, total= 1.9min
[CV] clf__estimator__n_neighbors=5 ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.8min remaining:    0.0s


[CV] .................... clf__estimator__n_neighbors=5, total= 2.0min
[CV] clf__estimator__n_neighbors=5 ...................................
[CV] .................... clf__estimator__n_neighbors=5, total= 1.9min
[CV] clf__estimator__n_neighbors=50 ..................................
[CV] ................... clf__estimator__n_neighbors=50, total= 2.0min
[CV] clf__estimator__n_neighbors=50 ..................................
[CV] ................... clf__estimator__n_neighbors=50, total= 2.0min
[CV] clf__estimator__n_neighbors=50 ..................................
[CV] ................... clf__estimator__n_neighbors=50, total= 1.9min
[CV] clf__estimator__n_neighbors=100 .................................
[CV] .................. clf__estimator__n_neighbors=100, total= 2.0min
[CV] clf__estimator__n_neighbors=100 .................................
[CV] .................. clf__estimator__n_neighbors=100, total= 2.0min
[CV] clf__estimator__n_neighbors=100 .................................
[CV] .

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 52.3min finished



 related 

             precision    recall  f1-score   support

          0       0.73      0.27      0.39      1245
          1       0.81      0.97      0.88      3998

avg / total       0.79      0.80      0.77      5243


 request 

             precision    recall  f1-score   support

          0       0.97      0.10      0.19      4352
          1       0.18      0.98      0.31       891

avg / total       0.83      0.25      0.21      5243


 offer 

             precision    recall  f1-score   support

          0       1.00      0.09      0.16      5219
          1       0.01      1.00      0.01        24

avg / total       1.00      0.09      0.16      5243


 aid_related 

             precision    recall  f1-score   support

          0       0.94      0.14      0.25      3079
          1       0.45      0.99      0.62      2164

avg / total       0.74      0.49      0.40      5243


 medical_help 

             precision    recall  f1-score   support

          0       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
pipeline_rf = Pipeline([
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

parameters = {"clf__estimator__n_estimators": [10,50,100]}

cv = GridSearchCV(pipeline_rf, param_grid=parameters)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

display_results(y_test, y_pred)


 related 

             precision    recall  f1-score   support

          0       0.70      0.41      0.52      1245
          1       0.84      0.94      0.89      3998

avg / total       0.80      0.82      0.80      5243


 request 

             precision    recall  f1-score   support

          0       0.97      0.16      0.28      4352
          1       0.19      0.97      0.32       891

avg / total       0.84      0.30      0.29      5243


 offer 

             precision    recall  f1-score   support

          0       1.00      0.14      0.25      5219
          1       0.01      0.96      0.01        24

avg / total       0.99      0.14      0.24      5243


 aid_related 

             precision    recall  f1-score   support

          0       0.96      0.23      0.37      3079
          1       0.47      0.99      0.64      2164

avg / total       0.76      0.54      0.48      5243


 medical_help 

             precision    recall  f1-score   support

          0       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [26]:
# Trying other transformers
# through experimentation only NounCount improved performance

class NounCount(BaseEstimator, TransformerMixin):
    
    def count_nouns(self, text):
        list_of_noun_tags = ["NN", "NNP", "NNPS", "NNS"]
        noun_count = 0
        for word, tag in pos_tag(tokenize(text)):
            if tag in list_of_noun_tags:
                noun_count += 1
        return noun_count
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        text_transformed = pd.Series(X).apply(self.count_nouns)
        return pd.DataFrame(text_transformed)
    
class MessageLength(BaseEstimator, TransformerMixin):
    
    def count_nouns(self, text):
        return len(text)
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        text_transformed = pd.Series(X).apply(self.count_nouns)
        return pd.DataFrame(text_transformed)
    
class NumberCount(BaseEstimator, TransformerMixin):
    
    def count_nouns(self, text):
        return len(re.sub("\D", "",text))
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        text_transformed = pd.Series(X).apply(self.count_nouns)
        return pd.DataFrame(text_transformed)

In [27]:
def model_pipeline():
    
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            
            ('count_noun', NounCount())
        ])),

        ('scale', StandardScaler(with_mean=False)),
        ('clf', RandomForestClassifier(n_estimators=10, n_jobs=-1))
    ])
    
    return pipeline

In [28]:
model = model_pipeline().fit(X_train, y_train)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
display_results(y_test, y_pred)

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/lib/nltk_data'
**********************************************************************


### 9. Export your model as a pickle file

In [ ]:
import pickle
model = pickle.dumps(model)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
%matplotlib inline

import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk import pos_tag
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

import pickle

class NounCount(BaseEstimator, TransformerMixin):
    """
    Input: Inherits from the sklean.base class
    Creates: A transformer object that can be used to count the number of nouns in a corpus of documents
    """
    
    def count_nouns(self, text):
        list_of_noun_tags = ["NN", "NNP", "NNPS", "NNS"]
        noun_count = 0
        for word, tag in pos_tag(tokenize(text)):
            if tag in list_of_noun_tags:
                noun_count += 1
        return noun_count
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        text_transformed = pd.Series(X).apply(self.count_nouns)
        return pd.DataFrame(text_transformed)

def load_data(db, table):
    """
    Inputs: The database and table name.
    Outputs: The data split into train and test data for the model.
    """
    
    engine = create_engine(db)
    df = pd.read_sql_table(table, engine)  
    X = df.message
    Y = df.drop(labels=["id","message","original","genre"],axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

def tokenize(text):
    """
    Inputs: A singel document of unedited text.
    Outputs: A single document of tokenized and standardized text.
    """
    # remove punctuation and tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    token = tokenizer.tokenize(text.lower())

    # remove stop words:
    stopwords_english = stopwords.words("english")
    tokens = [word for word in token if word not in stopwords_english]
    
    return tokens
        
def model_pipeline():
    """
    Inputs: None.
    Outputs: Creates a data pipeline that transforms the data into a TF-IDF Vectorized form,
             while in parallel creates a new feature that counts the nouns. The pipeline 
             then merges these together and scales the data before using it with a
             RandomForeset Classifier. 
    """
    
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            
            ('count_noun', NounCount())
        ])),

        ('scale', StandardScaler(with_mean=False)),
        ('clf', RandomForestClassifier(n_jobs=-1))
    ])
    
    parameters = {"clf__n_estimators":[10,50,100]}
    model = GridSearchCV(pipeline, param_grid=parameters, verbose=1)
    
    return model

def display_results(y_true, y_pred):
    """
    Inputs: labelled data, the true value of Y data, and the predicted value of Y data.
    Returns: the classification report for all the labels.
    """
    for index, column in enumerate(y_true):
        print(column, classification_report(y_true[column], y_pred[:, index]))
        
def save_model(model, filename):
    pickle.dump(model, open(filename, "wb"))
        
def main(*args):
    """
    Inputs: The database and sql table name.
    Outputs: 
    """
    if len(args) == 3:
        X_train, X_test, y_train, y_test = load_data(args[0], args[1])
        print("\n Data loaded successfully \n")
        model = model_pipeline()
        print("\n Pipeline created successfully \n")
        model.fit(X_train, y_train)
        print("\n Model trained successfully \n")
        y_pred = model.predict(X_test)
        print("\n Model Prediction completed \n")
        display_results(y_test, y_pred)
        print("\n Displayed results successfully \n")
        save_model(model, args[2])
        print("\n Saved model successfully \n")
    else:
        print(f"\n Not enough inputs (total of {3-len(args)} missing) were given to the programme. \n",
             "Please input the arguments in the following order:\n",
             "database directory, name of SQL table, save name for the machine learning model.")
        
if __name__ == '__main__':
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.

 Not enough inputs (total of 3 missing) were given to the programme. 
 Please input the arguments in the following order:
 database directory, name of SQL table, save name for the machine learning model.


In [2]:
main("sqlite:///InsertDatabaseName.db", "DisasterResponse", "model")


 Data loaded successfully 


 Pipeline created successfully 

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 20.8min finished



 Model trained successfully 


 Model Prediction completed 

related              precision    recall  f1-score   support

          0       0.67      0.53      0.59      1245
          1       0.86      0.92      0.89      3998

avg / total       0.82      0.83      0.82      5243

request              precision    recall  f1-score   support

          0       0.90      0.98      0.94      4352
          1       0.84      0.48      0.61       891

avg / total       0.89      0.90      0.88      5243

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5219
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5243

aid_related              precision    recall  f1-score   support

          0       0.74      0.92      0.82      3079
          1       0.82      0.54      0.65      2164

avg / total       0.77      0.76      0.75      5243

medical_help              precision    recal

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 Saved model successfully 

